## 課題：武蔵野大学 Web サイトのサイトマップ抽出

- 目的：武蔵野大学のトップページ（https://www.musashino-u.ac.jp/）にアクセスし、同一ドメイン内の全リンクを辿って各ページの URL と <title> を収集し、辞書（key=URL、value=<title>）に格納して print() で表示する。
- 実行上の制約：必ず time.sleep(seconds) を使ってサーバー負荷を軽減すること。

GitHub フローの手順（ローカルでの操作例）：

```bash
# 1. main (または upstream の最新) を取得
git checkout main
git pull origin main

# 2. 新しい作業ブランチを作る
git checkout -b feat/scrape-musashino-sitemap

# 3. ノートブックを編集・保存（この assignment.ipynb を編集）
# 4. 変更をステージしてコミット
git add assignment.ipynb
git commit -m "Add crawler notebook to extract musashino site titles"

# 5. ブランチをリモートに push
git push -u origin feat/scrape-musashino-sitemap

# 6. GitHub で Pull Request を作成してレビュー・マージ
```

注意：ノートブックは差分が大きくなりがちです。必要に応じて出力を消してコミットするか、`.gitattributes` でノートブックの扱いを調整してください。

In [1]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin, urlparse
import pandas as pd

# 武蔵野大学のトップページ
start_url = "https://www.musashino-u.ac.jp/"

# サイトマップを格納する辞書 {URL: title}
sitemap = {}

# 訪問済みURLを記録
visited = set()

def scrape(url):
    """指定URLの<title>と内部リンクを抽出する関数"""
    try:
        res = requests.get(url, timeout=10)
        res.encoding = res.apparent_encoding
        soup = BeautifulSoup(res.text, "html.parser")

        # タイトルを取得
        title = soup.title.string.strip() if soup.title else "（タイトルなし）"
        sitemap[url] = title

        # 内部リンクを収集
        links = []
        for a in soup.find_all("a", href=True):
            href = a["href"].strip()
            full_url = urljoin(url, href)

            # 同一ドメインかつ不要な拡張子でないものを対象
            if urlparse(full_url).netloc == urlparse(start_url).netloc:
                if not full_url.lower().endswith((
                    ".pdf", ".jpg", ".jpeg", ".png", ".gif", ".mp4", ".zip"
                )):
                    if full_url not in visited:
                        links.append(full_url)
        return links

    except Exception as e:
        print(f"⚠️ エラー: {url} ({e})")
        return []

# クロール開始
to_visit = [start_url]

while to_visit:
    url = to_visit.pop(0)
    if url in visited:
        continue

    print(f"アクセス中: {url}")
    visited.add(url)

    new_links = scrape(url)
    time.sleep(0.75)  # サーバー負荷軽減（1秒間隔）

    # 新しい内部リンクを追加
    for link in new_links:
        if link not in visited and link not in to_visit:
            to_visit.append(link)

# 結果表示
print("\n=== 抽出結果 ===")
for u, t in sitemap.items():
    print(f"{u} : {t}")

# CSVとして保存
df = pd.DataFrame(list(sitemap.items()), columns=["URL", "Title"])
df.to_csv("musashino_sitemap.csv", index=False, encoding="utf-8-sig")

print("\n✅ CSVファイル 'musashino_sitemap.csv' に保存しました。")


アクセス中: https://www.musashino-u.ac.jp/
アクセス中: https://www.musashino-u.ac.jp/#main
アクセス中: https://www.musashino-u.ac.jp/access.html
アクセス中: https://www.musashino-u.ac.jp/admission/request.html
アクセス中: https://www.musashino-u.ac.jp/contact.html
アクセス中: https://www.musashino-u.ac.jp/prospective-students.html
アクセス中: https://www.musashino-u.ac.jp/students.html
アクセス中: https://www.musashino-u.ac.jp/alumni.html
アクセス中: https://www.musashino-u.ac.jp/parents.html
アクセス中: https://www.musashino-u.ac.jp/business.html
アクセス中: https://www.musashino-u.ac.jp/guide/
アクセス中: https://www.musashino-u.ac.jp/guide/profile/
アクセス中: https://www.musashino-u.ac.jp/guide/activities/
アクセス中: https://www.musashino-u.ac.jp/guide/campus/
アクセス中: https://www.musashino-u.ac.jp/guide/facility/
アクセス中: https://www.musashino-u.ac.jp/guide/information/
アクセス中: https://www.musashino-u.ac.jp/guide/profile/media/
アクセス中: https://www.musashino-u.ac.jp/admission/
アクセス中: https://www.musashino-u.ac.jp/admission/faculty/
アクセス中: https://www.musa

KeyboardInterrupt: 